In [ ]:
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
import seaborn as sns

from comet_ml import Experiment
from comet_ml.integration.sklearn import log_model

from dotenv import load_dotenv, find_dotenv
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn import metrics 
from sklearn.metrics import roc_curve, auc

import warnings
warnings.filterwarnings('ignore')

import pickle
load_dotenv(find_dotenv())

COMET_API_KEY = os.environ.get("COMET_API_KEY")
from comet_ml.api import API
import sys

sys.path.append('/Users/iamyash/Desktop/NHL-DataScience-lIFT-6758')  # Add the root project directory path
api = API() 

from src.utils.utils import *
from src.utils.plot_graphe import *
from src.utils.result_plot_utils import *

from src.utils.result_plot_utils import plot_calibration_multi, plot_cumu_goal_multi, plot_goal_ratio_multi, plot_roc_multi

# Playoffs test results

In [ ]:
test_df = pd.read_csv("../../data/FE2/test_P.csv")
len(test_df)

## baselines

In [ ]:
api.download_registry_model("ift6758-milestone2-udem", "baseline_model_angle", version="1.0.0", output_path="../../models", expand=True, stage=None)
with open("../../models/baseline_model_angle.pkl", 'rb') as m:
    baseline_model_angle = pickle.load(m)
    
baseline_angle_preds = baseline_model_angle.predict_proba(test_df[['angle']].fillna(test_df['angle'].mode()))

api.download_registry_model("ift6758-milestone2-udem", "baseline_model_ditance", version="1.0.0", output_path="../../models", expand=True, stage=None)
with open("../../models/baseline_model_ditance.pkl", 'rb') as m:
    baseline_model_distance = pickle.load(m)
baseline_dist_preds = baseline_model_distance.predict_proba(test_df[['distance']].fillna(test_df['distance'].mode()))

api.download_registry_model("ift6758-milestone2-udem", "baseline_model_distance-angle", version="1.0.0", output_path="../../models", expand=True, stage=None)
with open("../../models/baseline_model_distance+angle.pkl", 'rb') as m:
    baseline_model_distance_angle = pickle.load(m)
baseline_dist_angle_preds = baseline_model_distance_angle.predict_proba(test_df[['distance', 'angle']].fillna(test_df[['distance', 'angle']].mode()))
 

## Random Forest

## XGBoost

In [ ]:
api.download_registry_model("ift6758-milestone2-udem", "xgboost_3", version="1.0.0", output_path="../../models", expand=True, stage=None)
with open("../../models/XGBoost_3.pkl", 'rb') as m:
    xg_model = pickle.load(m)

features_selected = ['y',
 'speed',
 'shotType_Snap Shot',
 'isRebound',
 'shotType_Wrap-around',
 'game_time',
 'shotType_Wrist Shot',
 'eventPeriod',
 'timeFromLastEvent',
 'shotType_Slap Shot',
 'shotType_Backhand',
 'prevEventType_HIT',
 'angle',
 'prevEventType_STOP',
 'distance',
 ]

test_oneHot = pre_processing(test_df ,oneHot=True)
X, y = get_X_y(test_oneHot)
cols_when_model_builds = xg_model.get_booster().feature_names
X_test_xg = X[cols_when_model_builds]

preds_xg = xg_model.predict_proba(X[cols_when_model_builds])

In [ ]:
models = [('Model:Distance only', baseline_dist_preds[:, 1]), ('Model:Angle only', baseline_angle_preds[:, 1]), ('Model:Distance+Angle', baseline_dist_angle_preds[:, 1]), ('Model:XGBoost(Best)', preds_xg[:,1])]


plot_roc_multi(models, test_df[['isGoal']].values.reshape(-1))
plot_calibration_multi(models, test_df[['isGoal']].values.reshape(-1))
plot_goal_ratio_multi(models, test_df[['isGoal']].values.reshape(-1))
plot_cumu_goal_multi(models, test_df[['isGoal']].values.reshape(-1))


## Regular Test results

In [ ]:
test_df = pd.read_csv("../../data/FE2/test_R.csv")
len(test_df)

## Baselines

In [ ]:
api.download_registry_model("ift6758-milestone2-udem", "baseline_model_angle", version="1.0.0", output_path="../../models", expand=True, stage=None)
with open("../../models/baseline_model_angle.pkl", 'rb') as m:
    baseline_model_angle = pickle.load(m)
    
baseline_angle_preds = baseline_model_angle.predict_proba(test_df[['angle']].fillna(test_df['angle'].mode()))

api.download_registry_model("ift6758-milestone2-udem", "baseline_model_ditance", version="1.0.0", output_path="../../models", expand=True, stage=None)
with open("../../models/baseline_model_ditance.pkl", 'rb') as m:
    baseline_model_distance = pickle.load(m)
baseline_dist_preds = baseline_model_distance.predict_proba(test_df[['distance']].fillna(test_df['distance'].mode()))

api.download_registry_model("ift6758-milestone2-udem", "baseline_model_distance-angle", version="1.0.0", output_path="../../models", expand=True, stage=None)
with open("../../models/baseline_model_distance+angle.pkl", 'rb') as m:
    baseline_model_distance_angle = pickle.load(m)
baseline_dist_angle_preds = baseline_model_distance_angle.predict_proba(test_df[['distance', 'angle']].fillna(test_df[['distance', 'angle']].mode()))
 

## Random forest

## XGBoost

In [ ]:
api.download_registry_model("ift6758-milestone2-udem", "xgboost_3", version="1.0.0", output_path="../../models", expand=True, stage=None)
with open("../../models/XGBoost_3.pkl", 'rb') as m:
    xg_model = pickle.load(m)

features_selected = ['y',
 'speed',
 'shotType_Snap Shot',
 'isRebound',
 'shotType_Wrap-around',
 'game_time',
 'shotType_Wrist Shot',
 'eventPeriod',
 'timeFromLastEvent',
 'shotType_Slap Shot',
 'shotType_Backhand',
 'prevEventType_HIT',
 'angle',
 'prevEventType_STOP',
 'distance',
 ]

test_oneHot = pre_processing(test_df ,oneHot=True)
X, y = get_X_y(test_oneHot)
cols_when_model_builds = xg_model.get_booster().feature_names
X_test_xg = X[cols_when_model_builds]

preds_xg = xg_model.predict_proba(X[cols_when_model_builds])

In [ ]:
models = [('Model:Distance only', baseline_dist_preds[:, 1]), ('Model:Angle only', baseline_angle_preds[:, 1]), ('Model:Distance+Angle', baseline_dist_angle_preds[:, 1]), ('Model:XGBoost(Best)', preds_xg[:,1])]


plot_roc_multi(models, test_df[['isGoal']].values.reshape(-1))
plot_calibration_multi(models, test_df[['isGoal']].values.reshape(-1))
plot_goal_ratio_multi(models, test_df[['isGoal']].values.reshape(-1))
plot_cumu_goal_multi(models, test_df[['isGoal']].values.reshape(-1))
